# Dados fictícios

In [ ]:
#Importando ferramentas básicas para construção de dados fictícios
import pandas as pd
import numpy as np

#Cria dados fictícios para exemplificar funcionamento do código
datas_ficticias = ['2022-08-01', '2022-08-01','2022-08-01','2022-08-01','2022-08-01','2022-08-01','2022-08-01','2022-08-02',
                   '2022-08-02','2022-08-02','2022-08-02','2022-08-02','2022-08-03','2022-08-03','2022-08-03','2022-08-03']

codigos_ficticios = ['cod01','cod98','cod84','cod34','cod22','cod17','cod68','cod09','cod53','cod33','cod41','cod26','cod89',
                     'cod93','cod06','cod67']

valores_ficticios = [100.34, 56.50, 90.55, 567.00, 1000.23, 123.32, 903.58, 42.00, 800.00, 800.00, 42.00, 350.00, 230.50,
                     202.10, 105.00, 300.00]

dados_ficticios = pd.DataFrame({'Data':datas_ficticias, 'Código':codigos_ficticios,
                                'Valor':valores_ficticios})

#Exibe dados fictícios
dados_ficticios

# Imports

In [ ]:
#Importa recursos do Colab para troca de recursos com o Drive
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Função atualiza_sheets

In [ ]:
#Função para editar sheets - apagando os dados do dia do relatório gerado e atualizando-os
def atualiza_sheets(sheet, worksheet, df, coluna_periodo, periodo_str):

  #Define a planilha (sheet) a ser atualizada
  sheets = gc.open(sheet)
  #Define a aba (worksheet) a ser atualizada
  sheets_page = sheets.worksheet(worksheet)
  #Cria um dataframe com os dados já existentes da aba a ser atualizada
  df_worksheet = pd.DataFrame(sheets_page.get_all_records())
  #Filtra o dataframe por períodos diferentes do periodo_str
  novo_df_worksheet = df_worksheet[df_worksheet[coluna_periodo] != periodo_str]
  #Limpa os dados da aba selecionada
  sheets_page.clear()
  #Concatena o dataframe filtrado da aba selecionada com o dataframe do dia em questão
  novo_df_worksheet = pd.concat([novo_df_worksheet, df])
  #Preenche os NaN com vazios (arquivos json não comportam dados nesse formato)
  novo_df_worksheet = novo_df_worksheet.fillna('')
  #Classifica o dataframe resultante pela coluna do período
  novo_df_worksheet.sort_values(by = coluna_periodo, ascending = True, inplace = True)
  #Atualiza a aba selecionada com o DataFrame resultante
  sheets_page.update([novo_df_worksheet.columns.values.tolist()] + novo_df_worksheet.values.tolist())

# Execução

In [ ]:
periodos = ['2022-08-01','2022-08-02','2022-08-03']

for periodo_str in periodos:

  #Seleciona, dentre os dados tratados, somente os do período de interesse
  dados_periodo = dados_ficticios[dados_ficticios['Data'] == periodo_str]

  #Execução da função atualiza_sheets
  atualiza_sheets('Exemplo atualiza_sheets','Dados_prévios',dados_periodo,'Data',periodo_str)